- **Module:** read_and_map_aeronet_interactive.ipynb
- **Authors:** Petar Grigorov, Alqamah Sayeed, and Pawan Gupta
- **Organization:** NASA AERONET (https://aeronet.gsfc.nasa.gov/)
- **Date:** 09/28/2023
- **Last Revision:** 07/29/2024
- **Purpose:** To access and map the AERONET data from Web API
- **Disclaimer:** The code is for demonstration purposes only. Users are responsible to check for accuracy and revise to fit their objective.
- **Contact:** Report any concern or question related to the code to pawan.gupta@nasa.gov or petar.t.grigorov@nasa.gov
- **Readme:** https://github.com/pawanpgupta/AERONET/blob/Python/README/Read_and_map_AERONET

**Required packages installation and importing**

In [ ]:
!pip uninstall -y numpy pandas shapely
!pip install numpy==1.26.4 pandas==2.0.3 shapely==1.8.5
!pip install cartopy
!pip install beautifulsoup4
!pip install requests
!pip install geopandas
!pip install ipywidgets

from bs4 import BeautifulSoup      #reads data from website (web scraping)
import re                          #regular expression matching operations (RegEx)
import requests                    #useful for sending HTTP requests using python
import shutil                      #useful for creating zip files
import numpy as np                 #for array manipulation
import datetime                    #for time data manipulation
import pandas as pd                #for data querying and processing
import geopandas as gpd            #same as pandas, but for geospatial data
import matplotlib.pyplot as plt    #for creating plots

import cartopy.crs as ccrs         #for creating geographical maps
import cartopy.feature as cfeature
from copy import deepcopy

from ipywidgets import interact, widgets  #for creating fun widgets
from IPython.display import display, HTML
import matplotlib.animation as animation

import warnings
warnings.filterwarnings('ignore')

from google.colab import drive      #imports local google drive
drive.mount('/drive')               #mounts local google drive onto colab

**Setup input parameters such as date, data level, averaging type, AOD range for mapping, AOD/Angstrom exponent, and geographical limits**

In [ ]:
dt_initial = '20240627'                 #starting date YYYYMMDD format
dt_final = '20240714'                   #final date YYYYMMDD format
level = 1.5                             #AERONET data level (1.0, 1.5 or 2.0)
average_type = 'daily'                  #Specifies data aggregation (daily or hourly)
vis_min = 0.0                           #any AOD/AE with smaller value will show as green on the color map, adjust as necessary
vis_max = 1.0                           #any AOD/AE with larger value will show as red on the color map, adjust as necessary
feature_choice = 2                      #Enter (1) if specifying an AOD wavelength or (2) if specifying an Angstrom exponent
wavelength = 500                        #Available choices: 1640, 1020, 870, 865, 779, 675, 667, 620, 560, 555, 551, 532, 531, 510, 500, 490, 443, 440, 412, 400, 380, 340
Angstrom_exp = '440-675'                #Available choices: '440-870','380-500','440-675','500-870','340-440','440-675(Polar)'
#Bounding box: Coordinates must be in decimal degrees (including decimal)
lat1,lon1 = 23.,-135.                   #lat1,lon1 - Lower Left
lat2,lon2 = 53.,-65.                    #lat2,lon2 - Upper Right

**Get desired AERONET data using web services, then scraping data from website**

In [ ]:
yr_initial = dt_initial[:4]               #initial year
mon_initial = dt_initial[4:6]             #initial month
day_initial = dt_initial[6:]              #initial day

yr_final = dt_final[:4]                   #final year
mon_final = dt_final[4:6]                 #final month
day_final = dt_final[6:]                  #final day

if level == 1 or level == 1.0:
  lev = 10
elif level == 1.5:
  lev = 15
elif level == 2 or level == 2.0:
  lev = 20
else:
  print("\nIncorrect input for data level type. Defaulting to level 1.5")
  lev = 15

if lev == 20 and int(yr_initial) == datetime.date.today().year:               #if user wants level 2 data for the current year, program alerts that data may not be available
  lev = 15                                                                      #defaults to level 1.5 data
  print("\nThere is no level 2 data available for the current year. Defaulting to level 1.5 data")

url = 'https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?lat1='+str(lat1)+'&lon1='+str(lon1)+'&lat2='+str(lat2)+'&lon2='+str(lon2)+'&year='+yr_initial+'&month='+mon_initial+'&day='+day_initial+'&year2='+yr_final+'&month2='+mon_final+'&day2='+day_final+'&AOD'+str(lev)+'=1&AVG=10'
soup = BeautifulSoup(requests.get(url).text) #web services contents are read here from URL

**Read and filter downloaded data as per user average type specification**

In [ ]:
with open(r'/content/temp.txt' ,"w") as oFile:          #writes the data scraped from "beautiful soup" to a text file on your local Google drive
    oFile.write(str(soup.text))
    oFile.close()

df = pd.read_csv(r'/content/temp.txt',skiprows = 5)     #loads the csv data into a Pandas dataframe
!rm temp.txt

if len(df) > 0:
  df = df.replace(-999.0, np.nan)                                     #replaces all -999.0 vakyes with NaN; helps with accurate data aggregation
  df.rename(columns={'Site_Latitude(Degrees)': 'Site_Latitude', 'Site_Longitude(Degrees)': 'Site_Longitude'}, inplace = True)
  df[['Day','Month','Year']] = df['Date(dd:mm:yyyy)'].str.split(':',expand=True)                                #splits the date column and then joins it back together using "-" instead of ":"
  df['Date'] = df[['Year','Month','Day']].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")       #because datetime format in python does not recognize colons
  df['Date']= pd.to_datetime(df['Date'])                              #converts the new date column to datetime format
  df['Hour'] = df['Time(hh:mm:ss)'].str[:2]                           #creates Hour column using just the HH component of the Time column
  numeric_cols = df.select_dtypes(include=['number']).columns         #defines the numeric columns, so aggregation functions do not crash with non-numeric ones.

  if average_type == 'daily':
    df = df.groupby(['AERONET_Site', 'Date'])[numeric_cols].mean()
  elif average_type == 'hourly':
    df = df.groupby(['AERONET_Site', 'Date','Hour'])[numeric_cols].mean()
  else:
    average_type = 'daily'
    df = df.groupby(['AERONET_Site', 'Date']).mean()
    print("\nIncorrect input for average type. Defaulting to daily averages.")

else:
  print("No data to parse. Please retry with different parameters.")

**AOD Wavelength or Angstrom Exponent Selection**

In [ ]:
AOD_col = [col for col in df.columns if 'AOD_' in col and 'nm' in col] #list of AOD columns, used for mapping user input to them
Ang_exp_col = [col for col in df.columns if 'Angstrom_Exponent' in col] #list of Angstrom Exponent columns, used for mapping user input to them

AOD_val = [int(re.search(r'\d+', col).group()) for col in AOD_col] #expected user input choices for AOD
Ang_exp_val = [item.split('_')[0] for item in Ang_exp_col] #expected user input choices for AE
Ang_exp_val[-1] += '(Polar)' #manually adds the polar channel to the list

if feature_choice == 1:
  if wavelength in AOD_val:             #if user input for AOD wavelength matches a value in the list, code proceeds forward. Otherwise it prompts user to try again
    for i in range(len(AOD_col)):
      if wavelength == AOD_val[i]:      #code scans the list of columns and list of possible values, and matches user input to the appropriate column name
        df = df[['Site_Latitude','Site_Longitude', AOD_col[i]]]         #if a match exists, the column name is matched to the actual column and it is then appended to the dataset
  else:
    df = df[['Site_Latitude','Site_Longitude','AOD_500nm']]
    print("\nInput for AOD wavelength is not in list. Defaulting to 500nm.")
elif feature_choice == 2:
  if Angstrom_exp in Ang_exp_val:     #if user input for Angstrom Exponent matches a value in the list, code proceeds forward. Otherwise it prompts user to try again
    for i in range(len(Ang_exp_col)):
      if Angstrom_exp == Ang_exp_val[i]:  #code scans the list of columns and list of possible values, and matches user input to the appropriate column nam
        df = df[['Site_Latitude','Site_Longitude', Ang_exp_col[i]]]     #if a match exists, the column name is matched to the actual column and it is then appended to the dataset
  else:
    df = df[['Site_Latitude','Site_Longitude','440-675']]
    print("\nInput for Angstrom Exponent is not in list. Defaulting to 440-675.")
else:
  feature_choice == 1
  print("\nIncorrect input for feature choice. Defaulting to AOD.")
  if wavelength in AOD_val:             #if user input for AOD wavelength matches a value in the list, code proceeds forward. Otherwise it prompts user to try again
    for i in range(len(AOD_col)):
      if wavelength == AOD_val[i]:      #code scans the list of columns and list of possible values, and matches user input to the appropriate column name
        df = df[['Site_Latitude','Site_Longitude', AOD_col[i]]]         #if a match exists, the column name is matched to the actual column and it is then appended to the dataset
  else:
    df = df[['Site_Latitude','Site_Longitude','AOD_500nm']]
    print("\nInput for AOD wavelength is not in list. Defaulting to 500nm.")

df = df.dropna() #Drops NaN or -999.0 values
df = df.reset_index() #resets the index
df

**Interactive Map - PlateCarree Projection**

In [ ]:
geo_df = deepcopy(df)
projection = ccrs.PlateCarree()
colbar = 'RdYlGn_r'

def plot_date(date_index):
    fig, ax = plt.subplots(figsize=(16, 12), subplot_kw={'projection': projection}, frameon=True)
    ax.set_extent([lon1,lon2,lat1,lat2],crs=ccrs.PlateCarree())

    countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    countries.plot(color="lightgrey", ax=ax, zorder=0, alpha=0.1)

    selected_date = date_list[date_index]
    selected_data = geo_df.loc[geo_df['Date'] == selected_date].reset_index(drop=True)

    colbar = plt.cm.get_cmap('RdYlGn_r')
    colbar.set_extremes(under='gray', over='magenta')
    cm = ax.scatter(x=selected_data["Site_Longitude"], y=selected_data["Site_Latitude"],
                    c=selected_data[selected_data.columns[-1]],
                    cmap=colbar, vmin=vis_min, vmax=vis_max, s=200, zorder=1)

    formatted_date = pd.to_datetime(selected_date).strftime('%Y-%m-%d')
    ax.set_title(formatted_date, size=20, weight='bold')
    ax.coastlines(resolution='10m', zorder=0)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)
    ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)

    cax = fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0, 0.02, ax.get_position().height])
    cbar = plt.colorbar(cm, cax=cax, extend='both')
    cax.set_ylabel(geo_df.columns[-1], size=20, weight='bold')
    tick_font_size = 20
    cbar.ax.tick_params(labelsize=tick_font_size)
    plt.show()

def plot_datetime(date_index, hour_index):
    selected_date = date_list[date_index]
    selected_hour = hour_list[hour_index]

    selected_data = geo_df[(geo_df['Date'] == selected_date) & (geo_df['Hour'] == selected_hour)].reset_index(drop=True)

    fig, ax = plt.subplots(figsize=(16, 12), subplot_kw={'projection': projection}, frameon=True)
    plt.xlim([lon1, lon2])
    plt.ylim([lat1, lat2])

    countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    countries.plot(color="lightgrey", ax=ax, zorder=0, alpha=0.1)

    colbar = plt.cm.get_cmap('RdYlGn_r')
    colbar.set_extremes(under='gray', over='magenta')
    cm = ax.scatter(x=selected_data["Site_Longitude"], y=selected_data["Site_Latitude"],
                    c=selected_data[selected_data.columns[-1]],
                    cmap=colbar, vmin=vis_min, vmax=vis_max, s=200, zorder=1)

    formatted_date = pd.to_datetime(selected_date).strftime('%Y-%m-%d')
    formatted_time = pd.to_datetime(selected_hour, format='%H').strftime('%H:%M:%S')
    ax.set_title(f'{formatted_date} {formatted_time} GMT', size=20, weight='bold')
    ax.coastlines(resolution='10m', zorder=0)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)
    ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)

    cax = fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0, 0.02, ax.get_position().height])
    cbar = plt.colorbar(cm, cax=cax, extend='both')
    cax.set_ylabel(geo_df.columns[-1], size=20, weight='bold')
    tick_font_size = 20
    cbar.ax.tick_params(labelsize=tick_font_size)
    plt.show()

def display_date_map(selected_date):
    date_index = date_labels.index(selected_date)
    plot_date(date_index)

def display_datetime_map(selected_date, selected_hour):
    date_index = date_labels.index(selected_date)
    hour_index = hour_labels.index(selected_hour)
    plot_datetime(date_index, hour_index)

if average_type == 'daily':
  date_list = sorted(geo_df['Date'].unique())
  date_labels = [pd.to_datetime(date).strftime('%Y-%m-%d') for date in date_list]
  date_slider = widgets.SelectionSlider(options=date_labels, description='Date')
  interact(display_date_map, selected_date=date_slider)

elif average_type == 'hourly':
  date_list = sorted(geo_df['Date'].unique())
  hour_list = sorted(geo_df['Hour'].unique())

  date_labels = [pd.to_datetime(date).strftime('%Y-%m-%d') for date in date_list]
  hour_labels = [datetime.datetime.strptime(hour_str, '%H').strftime("%H:%M:%S") for hour_str in hour_list]
  date_slider = widgets.SelectionSlider(options=date_labels, description='Date')
  hour_slider = widgets.SelectionSlider(options=hour_labels, description='Hour')

  interact(display_datetime_map, selected_date=date_slider, selected_hour=hour_slider)

elif average_type == 'total':
  fig, ax = plt.subplots(figsize=(16,12),subplot_kw={'projection': projection},frameon=True)
  countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
  countries.plot(color="lightgrey",ax=ax,zorder=0,alpha=0.1)
  ax.coastlines(resolution='10m',zorder=1)
  ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=1)
  ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=1)
  plt.xlim([lon1,lon2])
  plt.ylim([lat1,lat2])
  colbar = plt.cm.get_cmap('RdYlGn_r')
  colbar.set_extremes(under='gray',over='magenta')
  cm = ax.scatter(x=geo_df["Site_Longitude"], y=geo_df["Site_Latitude"],
                    c=geo_df[geo_df.columns[-1]],
                cmap=colbar, vmin = vis_min, vmax = vis_max, s = 200, zorder=1)
  ax.set_title("Site Average",size=20, weight='bold')
  cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
  cbar=plt.colorbar(cm, cax=cax, extend = 'both')
  cax.set_ylabel(geo_df.columns[-1], size=20, weight='bold')
  tick_font_size = 20
  cbar.ax.tick_params(labelsize=tick_font_size)
  plt.show()

**Interactive Map - Orthographic Projection**

In [ ]:
geo_df = deepcopy(df)

central_longitude = lon1 + abs(lon2 - lon2)//2
central_latitude  = lat1 + abs(lat1 - lat2)//2
projection=ccrs.Orthographic(central_longitude=central_longitude, central_latitude=central_latitude)
colbar='RdYlGn_r'

def plot_date(date_index):
    fig, ax = plt.subplots(figsize=(12, 9), subplot_kw={'projection': projection}, frameon=True)
    ax.set_extent([lon1, lon2, lat1, lat2], crs=ccrs.PlateCarree())

    countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    countries.plot(color="lightgrey", ax=ax, zorder=0, alpha=0.1)

    selected_date = date_list[date_index]
    selected_data = geo_df.loc[geo_df['Date'] == selected_date].reset_index(drop=True)

    colbar = plt.cm.get_cmap('RdYlGn_r')
    colbar.set_extremes(under='gray', over='magenta')
    cm = ax.scatter(x=selected_data["Site_Longitude"], y=selected_data["Site_Latitude"],
                    c=selected_data[selected_data.columns[-1]],
                    cmap=colbar, vmin=vis_min, vmax=vis_max, s=200, zorder=1,
                    transform=ccrs.PlateCarree())

    formatted_date = pd.to_datetime(selected_date).strftime('%Y-%m-%d')
    ax.set_title(formatted_date, size=20, weight='bold')
    ax.coastlines(resolution='10m', zorder=0)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)
    ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)

    cax = fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0, 0.02, ax.get_position().height])
    cbar = plt.colorbar(cm, cax=cax, extend='both')
    cax.set_ylabel(geo_df.columns[-1], size=20, weight='bold')
    tick_font_size = 20
    cbar.ax.tick_params(labelsize=tick_font_size)
    plt.show()

def plot_datetime(date_index, hour_index):
    selected_date = date_list[date_index]
    selected_hour = hour_list[hour_index]

    selected_data = geo_df[(geo_df['Date'] == selected_date) & (geo_df['Hour'] == selected_hour)].reset_index(drop=True)
    fig, ax = plt.subplots(figsize=(12, 9), subplot_kw={'projection': projection}, frameon=True)
    ax.set_extent([lon1, lon2, lat1, lat2], crs=ccrs.PlateCarree())

    countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    countries.plot(color="lightgrey", ax=ax, zorder=0, alpha=0.1)

    colbar = plt.cm.get_cmap('RdYlGn_r')
    colbar.set_extremes(under='gray', over='magenta')
    cm = ax.scatter(x=selected_data["Site_Longitude"], y=selected_data["Site_Latitude"],
                    c=selected_data[selected_data.columns[-1]],
                    cmap=colbar, vmin=vis_min, vmax=vis_max, s=200, zorder=1,
                    transform=ccrs.PlateCarree())

    formatted_date = pd.to_datetime(selected_date).strftime('%Y-%m-%d')
    formatted_time = pd.to_datetime(selected_hour, format='%H').strftime('%H:%M:%S')
    ax.set_title(f'{formatted_date} {formatted_time} GMT', size=20, weight='bold')
    ax.coastlines(resolution='10m', zorder=0)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)
    ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)

    cax = fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0, 0.02, ax.get_position().height])
    cbar = plt.colorbar(cm, cax=cax, extend='both')
    cax.set_ylabel(geo_df.columns[-1], size=20, weight='bold')
    tick_font_size = 20
    cbar.ax.tick_params(labelsize=tick_font_size)
    plt.show()

if average_type == 'daily':
  date_list = sorted(geo_df['Date'].unique())
  date_labels = [pd.to_datetime(date).strftime('%Y-%m-%d') for date in date_list]
  date_slider = widgets.SelectionSlider(options=date_labels, description='Date')
  interact(display_date_map, selected_date=date_slider)

elif average_type == 'hourly':
  date_list = sorted(geo_df['Date'].unique())
  hour_list = sorted(geo_df['Hour'].unique())

  date_labels = [pd.to_datetime(date).strftime('%Y-%m-%d') for date in date_list]
  hour_labels = [datetime.datetime.strptime(hour_str, '%H').strftime("%H:%M:%S") for hour_str in hour_list]
  date_slider = widgets.SelectionSlider(options=date_labels, description='Date')
  hour_slider = widgets.SelectionSlider(options=hour_labels, description='Hour')

  interact(display_datetime_map, selected_date=date_slider, selected_hour=hour_slider)

elif average_type == 'total':
  fig, ax = plt.subplots(figsize=(12,9),subplot_kw={'projection': projection},frameon=True)
  ax.set_extent([lon1,lon2,lat1,lat2],crs=ccrs.PlateCarree())

  countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
  countries.plot(color="lightgrey",ax=ax,zorder=0,alpha=0.1)

  colbar = plt.cm.get_cmap('RdYlGn_r')
  colbar.set_extremes(under='gray',over='magenta')
  cm = ax.scatter(x=geo_df["Site_Longitude"], y=geo_df["Site_Latitude"],
                    c=geo_df[geo_df.columns[-1]],
                cmap=colbar, vmin = vis_min, vmax = vis_max, s = 200, zorder=2, transform=ccrs.PlateCarree())

  ax.set_title("Site Averages",size=20, weight='bold')
  ax.coastlines(resolution='10m',zorder=1)
  ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=1)
  ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=1)

  cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
  plt.colorbar(cm, cax=cax, extend='both')
  cax.set_ylabel(geo_df.columns[-1], size=20, weight='bold')
  tick_font_size = 20
  cbar.ax.tick_params(labelsize=tick_font_size)
  plt.show()

**Animation - PlateCarree**

In [ ]:
geo_df = deepcopy(df)
projection = ccrs.PlateCarree()
colbar = 'RdYlGn_r'

if average_type == 'daily':
  geo_df['Timestamp'] = geo_df['Date'].astype(str)
  geo_df.insert(3, 'Timestamp', geo_df.pop('Timestamp'))
  date_list = np.unique(geo_df['Timestamp'])

if average_type == 'hourly':
  geo_df['Hour'] = pd.to_datetime(geo_df['Hour'].astype(str), format='%H')
  geo_df['Hour'] = geo_df['Hour'].dt.time
  geo_df['Date_Time'] = pd.to_datetime(geo_df['Date'].astype(str) + ' ' + geo_df['Hour'].astype(str))
  geo_df['Timestamp'] = geo_df[['Date_Time']].astype(str)
  geo_df = geo_df.drop(columns=['Date', 'Hour', 'Date_Time'])
  geo_df.insert(3, 'Timestamp', geo_df.pop('Timestamp'))
  date_list = np.unique(geo_df['Timestamp'])

fig, ax = plt.subplots(figsize=(14, 6), subplot_kw={'projection': projection}, frameon=True)
fig.subplots_adjust(left=0.05, right=0.9, top=0.95, bottom=0.05)
plt.close(fig)

def update_plot(frame):
    ax.clear()
    geo_df_frame = geo_df[geo_df['Timestamp'] == date_list[frame]].reset_index(drop=True)

    ax.set_xlim([lon1, lon2])
    ax.set_ylim([lat1, lat2])

    countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    countries.plot(color="lightgrey", ax=ax, zorder=0, alpha=0.1)

    colbar = plt.cm.get_cmap('RdYlGn_r')
    colbar.set_extremes(under='gray', over='magenta')
    sc = ax.scatter(x=geo_df_frame["Site_Longitude"], y=geo_df_frame["Site_Latitude"],
                    c=geo_df_frame[geo_df_frame.columns[-1]],
                    cmap=colbar, vmin=vis_min, vmax=vis_max, s=200, zorder=1)

    ax.set_title(date_list[frame], size=20, weight='bold')
    ax.coastlines(resolution='10m', zorder=0)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)
    ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)

    cax = fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0, 0.02, ax.get_position().height])
    cbar = plt.colorbar(sc, cax=cax, extend='both')
    cax.set_ylabel(geo_df_frame.columns[-1], size=20, weight='bold')
    tick_font_size = 20
    cbar.ax.tick_params(labelsize=tick_font_size)

ani = animation.FuncAnimation(fig, update_plot, frames=len(date_list), repeat=False)
out = display(HTML(""), display_id="animation")
out.update(HTML(ani.to_jshtml()))

**Animation - Orthographic**

In [ ]:
geo_df = deepcopy(df)
central_longitude = lon1 + abs (lon2 - lon2)//2
central_latitude  = lat1 + abs (lat1 - lat2)//2
projection=ccrs.Orthographic(central_longitude=central_longitude, central_latitude=central_latitude)
colbar='RdYlGn_r'

if average_type == 'daily':
  geo_df['Timestamp'] = geo_df['Date'].astype(str)
  geo_df.insert(3, 'Timestamp', geo_df.pop('Timestamp'))
  date_list = np.unique(geo_df['Timestamp'])

if average_type == 'hourly':
  geo_df['Hour'] = pd.to_datetime(geo_df['Hour'].astype(str), format='%H')
  geo_df['Hour'] = geo_df['Hour'].dt.time
  geo_df['Date_Time'] = pd.to_datetime(geo_df['Date'].astype(str) + ' ' + geo_df['Hour'].astype(str))
  geo_df['Timestamp'] = geo_df[['Date_Time']].astype(str)
  geo_df = geo_df.drop(columns=['Date', 'Hour', 'Date_Time'])
  geo_df.insert(3, 'Timestamp', geo_df.pop('Timestamp'))
  date_list = np.unique(geo_df['Timestamp'])

fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': projection}, frameon=True)
fig.subplots_adjust(left=0.05, right=0.9, top=0.95, bottom=0.05)
plt.close(fig)

def update_plot(frame):
    ax.clear()
    ax.set_extent([lon1,lon2,lat1,lat2],crs=ccrs.PlateCarree())
    geo_df_frame = geo_df[geo_df['Timestamp'] == date_list[frame]].reset_index(drop=True)

    countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    countries.plot(color="lightgrey", ax=ax, zorder=0, alpha=0.1)

    colbar = plt.cm.get_cmap('RdYlGn_r')
    colbar.set_extremes(under='gray', over='magenta')
    sc = ax.scatter(x=geo_df_frame["Site_Longitude"], y=geo_df_frame["Site_Latitude"],
                    c=geo_df_frame[geo_df_frame.columns[-1]],
                    cmap=colbar, vmin=vis_min, vmax=vis_max, s=200, zorder=1,transform=ccrs.PlateCarree())

    ax.set_title(date_list[frame], size=20, weight='bold')
    ax.coastlines(resolution='10m', zorder=0)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)
    ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray', zorder=0)

    cax = fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0, 0.02, ax.get_position().height])
    cbar = plt.colorbar(sc, cax=cax, extend='both')
    cax.set_ylabel(geo_df_frame.columns[-1], size=20, weight='bold')
    tick_font_size = 20
    cbar.ax.tick_params(labelsize=tick_font_size)

ani = animation.FuncAnimation(fig, update_plot, frames=len(date_list), repeat=False)
out = display(HTML(""), display_id="animation")
out.update(HTML(ani.to_jshtml()))